# 0. Install Dependencies

In [2]:
!pip install tensorflow==2.3.0 --user
!pip install gym
!pip install keras
!pip install keras-rl2

# 1. Test Random Environment with OpenAI Gym

In [3]:
!pip install gym[classic_control]

In [34]:
import gym 
import random

In [62]:
env = gym.make('CartPole-v1')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [63]:
actions

2

In [37]:
env.close()

In [38]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1])
        #env.step(action)
        n_state, reward, done, info, info2 = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:20.0
Episode:2 Score:15.0
Episode:3 Score:17.0
Episode:4 Score:16.0
Episode:5 Score:22.0
Episode:6 Score:29.0
Episode:7 Score:14.0
Episode:8 Score:29.0
Episode:9 Score:11.0
Episode:10 Score:39.0


# 2. Create a Deep Learning Model with Keras

In [39]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [71]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [72]:
model = build_model(states, actions)

In [73]:
states

4

In [74]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_12 (Dense)             (None, 24)                120       
_________________________________________________________________
dense_13 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [75]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [76]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [55]:
!pip install dopamine-rl --user

  Using cached dopamine_rl-4.0.6-py3-none-any.whl (179 kB)
  Using cached tf_slim-1.1.0-py2.py3-none-any.whl (352 kB)
  Using cached gin_config-0.5.0-py3-none-any.whl (61 kB)
  Using cached jax-0.2.17.tar.gz (693 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached opencv_python-4.6.0.66-cp36-abi3-win_amd64.whl (35.6 MB)
  Using cached dopamine_rl-4.0.5-py3-none-any.whl (174 kB)
  Using cached dopamine_rl-4.0.2-py3-none-any.whl (164 kB)
  Using cached dopamine_rl-4.0.1-py3-none-any.whl (151 kB)
  Using cached dopamine_rl-4.0.0-py3-none-any.whl (147 kB)
  Using cached dopamine_rl-3.2.1-py3-none-any.whl (127 kB)
  Using cached dopamine_rl-3.1.14-py3-none-any.whl (120 kB)
  Using cached dopamine_rl-3.1.13-py3-none-any.whl (119 kB)
  Using cached dopamine_rl-3.1.10-py3-none-any.whl (119 kB)
  Using cached dopamine_rl-3.1.9-py3-none-any.whl (118 kB)
  Using cached dopamine_rl-3.1.8-py3-none-any.whl (117 kB)
  Using cached d

In [77]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)


ValueError: Error when checking input: expected flatten_4_input to have shape (1, 4) but got array with shape (1, 2)

In [58]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...


ValueError: Error when checking input: expected flatten_2_input to have shape (1, 4) but got array with shape (1, 2)

# 4. Reloading Agent from Memory

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [20]:
del model
del dqn
del env

In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('dqn_weights.h5f')

In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)